In [19]:
import pandas as pd
import numpy as np
import cv2
import os

DIR = os.getcwd()

In [20]:
def readDataset(folder):
    images = []
    listFiles = os.listdir(folder)
    for file in listFiles:
        images.append(cv2.imread(folder+'/'+file,0))
    return np.array(images,dtype=object)

# Data Importation

In [21]:
realTrainData = readDataset(DIR+'/Datasets/dataset_train/real')
fakeTrainData = readDataset(DIR+'/Datasets/dataset_train/fake')
testData = readDataset(DIR+'/Datasets/dataset_test/dataset_test')

In [ ]:
fakeTrainData.head()

,id,category
0,image_13228.png,1
1,image_15881.png,1
2,image_14547.png,1
3,image_15659.png,1
4,image_12136.png,1


# Data Cleaning